In [1]:
!pip install pytorch-nlp


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 90 kB 7.1 MB/s 


In [2]:
#%%printcell
import torch
from torch.autograd import Variable
import numpy as np
import torch.functional as F
import torch.nn.functional as F
from torchnlp.datasets import imdb_dataset
import torch.nn as nn
import torch.optim as optim
import random
from torch.utils.data import Dataset, DataLoader

In [3]:
#hyper parameter
num_epoch = 100
learning_rate = 0.01
vector_size = 300

In [4]:
train = imdb_dataset(train=True)

aclImdb_v1.tar.gz: 84.1MB [00:09, 8.57MB/s]                            


In [5]:
from nltk import tokenize
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [6]:
text = ' '.join([s["text"] for s in train])
text = text.replace('<br /><br />',' ')
text = text.replace('\x85','')
text = text.lower()
from tqdm import tqdm

#sentence_tokenize = tokenize.sent_tokenize(text)
import re
tokenized_sentence = re.compile('[,.!?()]').split(text)
tokenized_corpus = [None]*len(tokenized_sentence)
for i in tqdm(range(len(tokenized_sentence))):
  tokenized_corpus[i] = tokenize.word_tokenize(tokenized_sentence[i])
#tokenized_corpus = [tokenize.word_tokenize(t) for t in tokenize.sent_tokenize(text)]

100%|██████████| 715693/715693 [00:59<00:00, 11989.87it/s]


In [7]:
del train
del text

In [8]:
vocabulary = []
for sentence in tqdm(tokenized_corpus):
    for token in sentence:
        #if token not in vocabulary:
            vocabulary.append(token)
vocabulary = list(set(vocabulary))

word2idx = {w: idx for (idx, w) in enumerate(vocabulary)}
idx2word = {idx: w for (idx, w) in enumerate(vocabulary)}

vocabulary_size = len(vocabulary)

100%|██████████| 715693/715693 [00:00<00:00, 862199.07it/s]


In [9]:
window_size = 2
pos_uv_list = [[] for _ in range(vocabulary_size)]
# for each sentence
for sentence in tqdm(tokenized_corpus):
    indices = [word2idx[word] for word in sentence]
    # for each word, threated as center word
    for center_word_pos in range(len(indices)):
        # for each window position
        for w in range(-window_size, window_size + 1):
            context_word_pos = center_word_pos + w
            # make soure not jump out sentence
            if context_word_pos < 0 or context_word_pos >= len(indices) or center_word_pos == context_word_pos:
                continue
            context_word_idx = indices[context_word_pos]
            pos_uv_list[indices[center_word_pos]].append(context_word_idx)

#idx_pairs = np.array(pos_u) # it will be useful to have this as numpy array

100%|██████████| 715693/715693 [00:16<00:00, 44681.91it/s]


In [10]:
vocabulary_size

99167

In [55]:
#neg_v = [list(range(vocabulary_size)) for _ in range(vocabulary_size)]
pos_u = []
pos_v = []
neg_v = []
for i in tqdm(range(vocabulary_size)):
    if pos_uv_list[i] != []:
        neg_set = set(range(vocabulary_size))
        neg_set.remove(i)
        neg_set = neg_set - set(pos_uv_list[i])
        neg_5 = np.random.choice(
            list(neg_set),
            size=(len(pos_uv_list[i]),5)
        )
        for j in range(len(pos_uv_list[i])):
            pos_u.append(i)
            pos_v.append(pos_uv_list[i][j])
            neg_v.append(neg_5[j])

100%|██████████| 99167/99167 [18:46<00:00, 87.99it/s]


In [63]:
pos_u = np.array(pos_u, dtype='int32')
pos_v = np.array(pos_v, dtype='int32')
neg_v = np.array(neg_v, dtype='int32')

In [58]:
indices = np.arange(pos_u.shape[0])
np.random.shuffle(indices)

pos_u = pos_u[indices]
pos_v = pos_v[indices]
neg_v = neg_v[indices]

In [64]:
class MyDataset(Dataset):
  def __init__(self, x, y, z):
    self.pos_u = x
    self.pos_v = y
    self.neg_v = z
    #self.shuffle_index = shuffle_index

  def __len__(self):
    return len(self.pos_u)
  
  def __getitem__(self, index):
    return self.pos_u[index], self.pos_v[index], self.neg_v[index]

In [76]:
dataset = MyDataset(pos_u,pos_v,neg_v)
train_loader = torch.utils.data.DataLoader(dataset, batch_size=1024, shuffle=False, num_workers=2)
#val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=512, shuffle=False, num_workers=2)
#vandeladorandom

In [66]:
class skipgramModel(nn.Module):
   def __init__(self, num_vocab, emb_dimension):
      super(skipgramModel, self).__init__()
      self.num_vocab = num_vocab
      self.emb_dimension = emb_dimension
      self.u_embeddings = nn.Embedding(num_vocab, emb_dimension, sparse=True)
      self.v_embeddings = nn.Embedding(num_vocab, emb_dimension, sparse=True)
      initrange = 0.5 / self.emb_dimension
      self.u_embeddings.weight.data.uniform_(-initrange, initrange)
      self.v_embeddings.weight.data.uniform_(-0, 0)
      
   def forward(self, pos_u, pos_v, neg_v):
      losses = []
      emb_u = self.u_embeddings(Variable(pos_u))
      emb_v = self.v_embeddings(Variable(pos_v))
      score = torch.mul(emb_u, emb_v).squeeze()
      score = torch.sum(score, dim=1)
      score = F.logsigmoid(score)
      losses.append(sum(score))
      neg_emb_v = self.v_embeddings(Variable(neg_v))
      neg_score = torch.bmm(neg_emb_v, emb_u.unsqueeze(2)).squeeze()
      neg_score = torch.sum(neg_score, dim=1)
      neg_score = F.logsigmoid(-1 * neg_score)
      losses.append(sum(neg_score))
      return -sum(losses)

model = skipgramModel(vocabulary_size, vector_size)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = model.to(device)
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

In [ ]:
model.train()
for epoch in range(20):
    print("\nStarting epoch {}".format(epoch+1))   
    total = 0
    running_loss = 0.0
    loader = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, data in loader:
        pos_u_in, pos_v_in, neg_v_in = data
        pos_u_in, pos_v_in, neg_v_in = pos_u_in.to(device).long(), pos_v_in.to(device).long(), neg_v_in.to(device).long()
        optimizer.zero_grad()
        loss = model(pos_u_in, pos_v_in, neg_v_in)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        total += 1
        loader.set_description("loss: {:.5f}".format(running_loss/total))

print('Finished Training')


Starting epoch 1


loss: 130.92816: 100%|██████████| 19808/19808 [12:43<00:00, 25.95it/s]


Starting epoch 2



loss: 117.25729: 100%|██████████| 19808/19808 [12:43<00:00, 25.93it/s]


Starting epoch 3


...

loss: 56.03010: 100%|██████████| 19808/19808 [12:39<00:00, 26.09it/s]


Starting epoch 17



loss: 53.86738: 100%|██████████| 19808/19808 [12:58<00:00, 25.44it/s]


Starting epoch 18



loss: 51.91578:  91%|█████████ | 18071/19808 [11:38<01:06, 26.24it/s]

In [ ]:
dataset.shuffle_indexhhhmmmmmmmm

In [ ]:
import pickle
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
pickle.dump(model,open('/content/drive/MyDrive/NLP/w2v_300dim_negloss.p','wb'))

In [ ]:
model_load = pickle.load(open('/content/drive/MyDrive/NLP/w2v_300dim_negloss.p','rb'))

In [ ]:
model_load.embedding(torch.tensor([1,2]).to(device))